# 00 - Configuration

For these notebooks, we'll be using the 'environs' package from pip. This is only done to easily load the USER_NAME, EXPERIMENT_NAME, and other constant variables. This is NOT a requirement of using Azure ML

In [ ]:
%pip install environs

In [ ]:
import environs

e_vars = environs.Env()
e_vars.read_env('../workshop.env')

USER_NAME = e_vars.str("USER_NAME")
EXPERIMENT_NAME = e_vars.str('EXPERIMENT_NAME')
ENVIRONMENT_NAME = e_vars.str("ENVIRONMENT_NAME")
DATASET_NAME = e_vars.str("DATASET_NAME")
PULL_DOCKER = e_vars.bool("PULL_DOCKER_IMAGE", False)

if not USER_NAME:
    raise NotImplementedError("Please enter your username in the `.env` file and run this cell again.")

In [ ]:
PULL_DOCKER

# Retrieve Azure Machine Learning Workspace object

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

If you don't have a config file present where you're running this script, there are several ways to retrieve it. (Compute Instances have it present by default).

The SDK makes it easy to connect to the Workspace and save a config file for future runs.
```python
ws = Workspace(workspace_name='[ENTER WORKSPACE NAME]',
               resource_group='[RESOURCE GROUP]',
               subscription_id='[SUBSCRIPTION ID]')

ws.write_config()
```

## Attach Data Store(s)
Typically, this only needs to be done once per datastore, per workspace

In [ ]:
BLOB_CONTAINER = os.environ.get("BLOB_CONTAINER_NAME", "diabetes")
BLOB_ACCOUNT = os.environ.get("BLOB_ACCT_NAME", "publicmldatasc")
BLOB_SAS_TOKEN = os.environ.get("BLOB_SAS_TOKEN", "?si=DiabetesReadOnly&sv=2019-10-10&sr=c&sig=cz9P%2B1V1eC6FvDIKBQNmA5nWqbsGfkqzdPTTYmiidfg%3D")

In [ ]:
datastore_name = 'diabetes'

from azureml.core import Datastore

if ws.datastores.get(datastore_name):
    print(f"'{datastore_name}' datastore found. Skipping registration")
else:
    print(f"'{datastore_name}' datastore not found. Registering with Workspace")
    _ = Datastore.register_azure_blob_container(workspace=ws,
                                                datastore_name='diabetes',
                                                container_name=BLOB_CONTAINER,
                                                account_name=BLOB_ACCOUNT,
                                                sas_token=BLOB_SAS_TOKEN)

In [ ]:
ws.datastores

In [ ]:
ds = ws.datastores['diabetes']

---
### Create and Register a Tabular Dataset
Create a 'Tabular' Dataset that can be used across AML

In [ ]:
from azureml.core import Dataset

dataset = Dataset.Tabular.from_parquet_files(path=(ds, 'parquet/diabetes.parquet'))
dataset = dataset.register(ws, name=DATASET_NAME)

---
## Create Azure Machine Learning Environment
AML Environments allow you to create, track, and version reusable environments to use across Azure Machine Learning. In this example, we'll be creating an AML Environment that leverages Docker to run 

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

In [ ]:
environ = Environment(ENVIRONMENT_NAME)

In [ ]:
# Set Docker Environment Settings
environ.docker.enabled = True

In [ ]:
# Set Python Environment Settings

cd = CondaDependencies.create(pip_packages=['azureml-defaults==1.7.0'], 
                              conda_packages=['scikit-learn==0.22.1', 
                                              'seaborn==0.10.1',
                                              'pandas==1.0.3',
                                              'pyarrow==0.15.1'],
                             python_version='3.8.3'
                             )

environ.python.conda_dependencies = cd

#### Build the Environment __OPTIONAL__
Here, we explicitly build the environment that we'll be using in the lab. If you choose not to build the environment at this point, the first time you run it, the environment will be built.

In [ ]:
environ = environ.register(ws)
build = environ.build(ws)

In [ ]:
build.wait_for_completion(show_output=True)

In [ ]:
if PULL_DOCKER:
    from docker import DockerClient

    docker = DockerClient.from_env()

    image_details = environ.get_image_details(ws)['dockerImage']
    docker.login(username=image_details['registry']['username'], 
                 password=image_details['registry']['password'], 
                 registry=image_details['registry']['address'])

    image = docker.images.pull(repository=image_details['registry']['address']+'/'+image_details['name'])[0]
    image.tag(image_details['name'])

<br><br><br><br><br>






###### Copyright (c) Microsoft Corporation. All rights reserved.  
###### Licensed under the MIT License.